In [ ]:
%matplotlib inline

from networkx.algorithms import connectivity as conn
from matplotlib import pyplot as plt
from tools import plot
import matplotlib.colors as mcolors
from tools import utils
import networkx as nx
import numpy as np
import time 
import os
import glob
import json

os.chdir("D:\Documenti\Lightning_Network_Logs\\recent_logs")
distance_list = [None] * len(glob.glob("*json"))
k_connectivity_list = [None] * len(glob.glob("*json"))
# Sets the picture size to a proper dimension.
plt.rcParams['figure.figsize'] = (10, 6)

# Create an array of snapshots of the network
graph_list = utils.load_graph_list("D:\Documenti\Lightning_Network_Logs\\recent_logs")

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plot.plot_average_eccentricity(graph_list)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plot.plot_average_degree(graph_list)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plot.plot_number_of_nodes(graph_list)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plot.plot_number_of_edges(graph_list)

In [ ]:
for i, graph in enumerate(graph_list):
    distance_list[i] = nx.diameter(graph)
plt.rcParams['figure.figsize'] = (10, 6)
plt.plot(distance_list)
plt.ylabel('Distance')
plt.savefig('distance.png', bbox_inches='tight')
plt.show()

In [ ]:
full_day_graph_list = utils.load_graph_list(r'D:\Documenti\Lightning_Network_Logs\recent_logs\full_day')

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plot.plot_number_of_nodes(full_day_graph_list, step=10, filename='daily_number_of_nodes.png')

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plot.plot_number_of_edges(full_day_graph_list, step=10, filename='daily_number_of_edges.png')


In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plot.plot_average_degree(full_day_graph_list, step=10, filename='daily_average_degree.png')

Calculate the Betweenness Centrality measures for all the snapshot of the network. Returns the subgraph of those network with the most important nodes (BC value greater than 3 times the mean) along with the values returned by the betweenness centality.

Then plots the percentage of important nodes (and their respective edges) wrt the total number of nodes and edges.
**Average duration for 15 snapshots: 5 mins.**

In [ ]:
betweenness_centrality_list = [nx.betweenness_centrality(graph) for graph in graph_list]

In [ ]:
daily_graph_list = utils.load_graph_list(r'D:\Documenti\Lightning_Network_Logs\recent_logs\daily')

utils.betweenness_centrality_intersection(betweenness_centrality_list)
number_of_snapshot = len(graph_list)

daily_betweenness_list = [nx.betweenness_centrality(graph) for graph in daily_graph_list]

# Finds the most important nodes. The evaluation is performed by aggregating the values for all the nodes and selecting
# the most important ones.        
betweenness_centrality_sums = betweenness_centrality_list[0].copy()
for i in range(1, len(graph_list)):
    for key in betweenness_centrality_sums.keys():
        betweenness_centrality_sums[key] = betweenness_centrality_sums[key] + betweenness_centrality_list[i][key]

# Sort dictionary of the sums of the betweenness centralities and sort it on their value.
sorted_list = sorted(betweenness_centrality_sums.items(), key=lambda x: x[1], reverse=True)
# Then select the top five nodes of the sorted list.
top_five = [sorted_list[i][0] for i in range(5)]
# For each snapshot of betweenness centrality, select the top 5 nodes and their values.
top_five_list = dict()
for item in top_five:
    top_five_list[item] = [daily_betweenness_list[i][item] for i in range(len(daily_graph_list))]

plt.rcParams['figure.figsize'] = (10, 6)
plt.xticks(range(number_of_snapshot))
for key in top_five_list.keys():
    plt.plot(top_five_list[key], label=key[0: 5])
plt.legend()
plt.savefig('daily_betweenness_centrality.png', bbox_inches='tight')
plt.show()


In [ ]:
# Intersection of the nodes that persist for all the snapshots
utils.betweenness_centrality_intersection(betweenness_centrality_list)
number_of_snapshot = len(betweenness_centrality_list)

# Finds the most important nodes. The evaluation is performed by aggregating the values for all the nodes and selecting
# the most important ones.k_co
betweenness_centrality_sums = betweenness_centrality_list[0].copy()
for i in range(1, number_of_snapshot):
    for key in betweenness_centrality_sums.keys():
        betweenness_centrality_sums[key] = betweenness_centrality_sums[key] + betweenness_centrality_list[i][key]

# Sort dictionary of the sums of the betweenness centralities and sort it on their value.
sorted_list = sorted(betweenness_centrality_sums.items(), key=lambda x: x[1], reverse=True)
# Then select the top five nodes of the sorted list.
top_five = [sorted_list[i][0] for i in range(5)]
# For each snapshot of betweenness centrality, select the top 5 nodes and their values.
top_five_list = dict()
for item in top_five:
    top_five_list[item] = [betweenness_centrality_list[i][item] for i in range(number_of_snapshot)]

plt.rcParams['figure.figsize'] = (10, 6)
plt.xticks(range(number_of_snapshot))
for key in top_five_list.keys():
    plt.plot(top_five_list[key], label=key[0: 5])
plt.legend()
os.chdir("D:\Documenti\Lightning_Network_Logs\\recent_logs")
plt.savefig('weekly_betweenness_centrality.png', bbox_inches='tight')
plt.show()

In [ ]:
import collections as ct

gap_dictionary = ct.defaultdict(list)
# Builds a presence list among the top 15 nodes over the period observed
presence_dict = utils.build_presence_dictionary(betweenness_centrality_list)
plt.rcParams['figure.figsize'] = (10, 6)        
for item in presence_dict:
    start = predecessor = presence_dict[item][0]
    for current in presence_dict[item]:
        if current - predecessor > 1:
            gap = predecessor - start
            gap_dictionary[item].append((start, gap))
            start = current
        predecessor = current
    gap = predecessor - start
    gap_dictionary[item].append((start, gap))

ax = plt.subplot()
ax.set_xlim(0, 15)
ax.set_xticks(range(15))
ax.set_xlabel('Snapshots')
ax.set_yticks(range(len(gap_dictionary.keys())))
ax.set_yticklabels([item[0: 5] for item in gap_dictionary.keys()])
for i, key in enumerate(gap_dictionary.keys()):
    for j, gap in enumerate(gap_dictionary[key]):
        if gap[1] == 0:
            gap_dictionary[key][j] = (gap[0], 1)
    ax.broken_barh(gap_dictionary[key], (i-0.5, 0.6), color='orange')
plt.savefig('gantt_chart.png', bbox_inches='tight')
plt.show()

Calculates k-vertex connectivity, this task is CPU intensive. Average of 5 mins per snapshot.

In [ ]:
new_path = r'D:\Documenti\Lightning_Network_Logs\recent_logs\k_conn'
if not os.path.exists(new_path):
    os.makedirs(new_path)
os.chdir(new_path)
start = time.time()
for i, graph in enumerate(graph_list): 
    k_connectivity_list[i] = conn.k_components(graph)
    end = time.time()
    for key in k_connectivity_list[i].keys():
        for j in range(len(k_connectivity_list[i][key])):
            k_connectivity_list[i][key][j] = list(k_connectivity_list[i][key][j])
    with open('k_conn_' + str(i) + '.json', 'w') as f:
        json.dump(k_connectivity_list[i], f, indent=4)
    elapsed = end - start
    print('Task number ' + str(i) + ' finished in ' + str(elapsed) + ' seconds.')

In [ ]:
# If the k-vertex connectivity has already been evaluated then run this cell
k_conn_path = r'D:\Documenti\Lightning_Network_Logs\recent_logs\k_conn'
k_connectivity_list = utils.read_k_conn_from_files(k_conn_path)

Calculates and plots the **average** vertex size for *k*-vertex components of the graph.

In [ ]:
import collections as ct

# If the k-vertex connectivity has already been evaluated then run this cell
k_conn_path = r'D:\Documenti\Lightning_Network_Logs\recent_logs\k_conn'
k_connectivity_list = utils.read_k_conn_from_files(k_conn_path)

k_vertex_size = dict()
plt.rcParams['figure.figsize'] = (10, 6)

# To calculate the average size of each component we need a counter that tells us
# how many time such a component exists.
# k_vertex_size[key] accumulates the size of each k_component.
# key_counter[key] accumulates how many times we have such a component.
key_counter = ct.defaultdict(int)
for i in range(len(k_connectivity_list)):
    for key in k_connectivity_list[i].keys():
        if i == 0:
            k_vertex_size[key] = len(k_connectivity_list[i][key][0])
        else:
            k_vertex_size[key] += len(k_connectivity_list[i][key][0])
        key_counter[key] += 1

# Inverted sorted list of number of nodes averages.
k_vertex_size = sorted([k_vertex_size[key] / key_counter[key] for key in k_vertex_size.keys()], reverse=True)
plt.xticks(np.arange(len(k_connectivity_list[0].keys())),
           sorted(np.arange(1, len(k_connectivity_list[0].keys()) + 1)))
for i, elem in enumerate(k_vertex_size):
    plt.bar(i, k_vertex_size[i], 0.7, align='center')
    
# Plot a blue dashed curve over the points of the size.
plt.plot(k_vertex_size, 'b--')

# Plot red dots over the spot identified by the list
plt.plot(k_vertex_size, 'or',)
plt.grid(True)
plt.savefig('k_connectivity_average_size.png', bbox_inches='tight')
plt.show()

In [ ]:
k_conn_path = r'D:\Documenti\Lightning_Network_Logs\recent_logs\k_conn'
k_connectivity_list = utils.read_k_conn_from_files(k_conn_path)
# Increase the size of the plot
plt.rcParams['figure.figsize'] = (10, 6)

betweenness_top_nodes = utils.get_betweenness_top_nodes(betweenness_centrality_list)
number_of_snapshot = len(k_connectivity_list)

# Select the max values of the keys for each snapshot, convert it to string.
max_keys = list()
for x in range(number_of_snapshot):
    max_keys.append(str(max([int(value) for value in k_connectivity_list[x].keys()])))

plt.yticks(np.arange(0, 51, 2))
for i in range(number_of_snapshot):
    intersect = k_connectivity_list[i][max_keys[i]][0] & betweenness_top_nodes
    plt.bar(i, len(k_connectivity_list[i][max_keys[i]][0]), 0.7, align='center', color='lightblue')
    plt.bar(i, len(intersect), 0.7, align='center', color='orange')
    plt.xticks(np.arange(number_of_snapshot), np.arange(1, number_of_snapshot+1))
plt.savefig('k_connectivity_top_fifteen_betweenness_centrality.png', bbox_inches='tight')
plt.show()

In [ ]:
start = time.time()
cutsets = [None] * len(graph_list)
for i, graph in enumerate(graph_list): 
    cutsets[i] = nx.all_node_cuts(graph)
    end = time.time()
    elapsed = end - start
    print('Task number ' + str(i) + ' finished in ' + str(elapsed) + ' seconds.')